## Granger Causality test

Il test di **causalità di Granger** è uno standard in letteratura. Nel nostro caso, serve a determinare se i valori passati di un indice GT sono utili per prevedere i valori futuri dell'inflazione, al di là di quanto l'inflazione possa essere prevista usando solo i suoi valori passati (e viceversa).

In [1]:
import pandas as pd
import numpy as np
import os
from statsmodels.tsa.stattools import grangercausalitytests

# --- CONFIGURAZIONE ---
# Path al file di input (output della Fase 2 - serie stazionarie)
PATH_INPUT_DIR_FASE2 = "/Users/tommaso/Desktop/tesi-inflation-gt/First_Difference_indexes/dati_preparati_fase2"
FILE_SERIE_STAZIONARIE_IN = os.path.join(PATH_INPUT_DIR_FASE2, "indici_gt_nic_stazionari_fase2.csv")

# Nomi delle colonne come salvate nello script precedente Fase 2
COL_INFLAZIONE_STAZ = 'NIC_destag_ISTAT_diff1'
COL_GT_INFLAZIONE_STAZ = 'indice_Inflazione_GT_PCA_SA_diff1'
COL_GT_TEMATICO_STAZ = 'indice_Tematico_GT_SA_diff1'

MAX_LAG_GRANGER = 16 # Numero massimo di ritardi da testare per la causalità di Granger.
                     # I lags si scelgono in base alla teoria, ai risultati CCF, o a criteri informativi.
                     # Dal CCF si nota che anche il lag 13 è significativo, dunque si considera fino a 16.
SIGNIFICANCE_LEVEL_GRANGER = 0.05

# --- FUNZIONI AUSILIARIE ---
def carica_dati_stazionari(path_file):
    """
    Carico le serie stazionarie (o trasformate) dal CSV salvato dalla Fase 2.
    """
    print(f"--- Caricamento Serie Stazionarie da: {path_file} ---")
    try:
        df = pd.read_csv(path_file, index_col=0)
        df.index = pd.to_datetime(df.index)
        print(f"Serie caricate con successo. Shape: {df.shape}")
        print("--- Fine Caricamento ---\n")
        return df
    except FileNotFoundError:
        print(f"ERRORE: File non trovato: {path_file}")
        return None
    except Exception as e:
        print(f"ERRORE durante il caricamento delle serie stazionarie: {e}")
        return None

# --- FUNZIONE PRINCIPALE PER IL TEST DI CAUSALITÀ DI GRANGER ---
def test_causalita_granger(data_df, variable_y, variable_x, max_lag_input, test_name="Granger Causality"):
    """
    Esegue il test di causalità di Granger tra due serie.
    Testa se variable_x "Granger-causa" variable_y.
    """
    print(f"--- Test di Causalità di Granger: {variable_x} -> {variable_y} (max_lag={max_lag_input}) ---")
    
    # Il test richiede un DataFrame con le due serie, senza NaN
    test_data = data_df[[variable_y, variable_x]].dropna()
    
    if test_data.empty or len(test_data) < max_lag_input + 20:
        print(f"Dati insufficienti per testare la causalità di Granger tra {variable_x} e {variable_y} con {max_lag_input} lags.")
        print("--- Fine Test Granger ---\n")
        return None
        
    try:
        # grangercausalitytests esegue il test per tutti i lag da 1 a max_lags
        # Restituisce un dizionario dove le chiavi sono i lag
        # Per ogni lag, esegue 4 test. Ci si concentra sul test F (ssr_ftest)
        results = grangercausalitytests(test_data, maxlag=max_lag_input, verbose=False)
        
        print(f"Risultati per {variable_x} Granger-causa {variable_y}:")
        causalita_trovata = False
        for lag_val in range(1, max_lag_input + 1):
            # Controllo se il lag esiste nei risultati (potrebbe non esistere se i dati sono troppo pochi per quel lag specifico)
            if lag_val in results:
                p_value_f_test = results[lag_val][0]['ssr_ftest'][1] 
                if p_value_f_test <= SIGNIFICANCE_LEVEL_GRANGER:
                    print(f"  Lag {lag_val}: Causalità SIGNIFICATIVA (p-value F-test: {p_value_f_test:.4f} <= {SIGNIFICANCE_LEVEL_GRANGER})")
                    causalita_trovata = True
                else:
                    print(f"  Lag {lag_val}: Causalità NON significativa (p-value F-test: {p_value_f_test:.4f} > {SIGNIFICANCE_LEVEL_GRANGER})")
            else:
                print(f"  Lag {lag_val}: Risultati non disponibili (probabilmente dati insufficienti per questo lag).")

        if causalita_trovata:
            print(f"CONCLUSIONE: Vi è evidenza che '{variable_x}' Granger-causa '{variable_y}' per almeno un lag testato.")
        else:
            print(f"CONCLUSIONE: Non vi è evidenza significativa che '{variable_x}' Granger-causa '{variable_y}' per i lag testati.")
            
        print("--- Fine Test Granger ---\n")
        return results
        
    except Exception as e:
        print(f"ERRORE durante il test di causalità di Granger per {variable_x} -> {variable_y}: {e}")
        print("Possibili cause: collinearità perfetta, dati insufficienti dopo dropna, problemi di specificazione.")
        print("--- Fine Test Granger ---\n")
        return None

# --- ESECUZIONE SCRIPT FASE 4 (Esplorativa - Causalità di Granger) ---
if __name__ == "__main__":
    print(">>> INIZIO SCRIPT FASE 4: Analisi di Causalità di Granger (Corretto) <<<\n")

    # 1. Carico le serie stazionarie (o trasformate) dalla Fase 2
    df_serie_stazionarie = carica_dati_stazionari(FILE_SERIE_STAZIONARIE_IN)

    if df_serie_stazionarie is None:
        print("ERRORE CRITICO: Impossibile caricare il file delle serie stazionarie. Script interrotto.")
        exit()

    # 2. Eseguo i test di causalità di Granger
    # Test 1: indice_Inflazione_GT_PCA_SA_diff1 -> NIC_destag_ISTAT_diff1
    if COL_GT_INFLAZIONE_STAZ in df_serie_stazionarie.columns and COL_INFLAZIONE_STAZ in df_serie_stazionarie.columns:
        print(f"\n*** Test di causalità per l'Indice GT Inflazione verso Inflazione NIC ***")
        test_causalita_granger(df_serie_stazionarie,
                               variable_y=COL_INFLAZIONE_STAZ,
                               variable_x=COL_GT_INFLAZIONE_STAZ,
                               max_lag_input=MAX_LAG_GRANGER)
    else:
        print(f"Skippo test Granger per '{COL_GT_INFLAZIONE_STAZ}' -> '{COL_INFLAZIONE_STAZ}' a causa di colonne mancanti.")

    # Test 2: NIC_destag_ISTAT_diff1 -> indice_Inflazione_GT_PCA_SA_diff1 (causalità inversa)
    if COL_GT_INFLAZIONE_STAZ in df_serie_stazionarie.columns and COL_INFLAZIONE_STAZ in df_serie_stazionarie.columns:
        print(f"\n*** Test di causalità inversa per l'Indice GT Inflazione (Inflazione NIC verso Indice GT) ***")
        test_causalita_granger(df_serie_stazionarie,
                               variable_y=COL_GT_INFLAZIONE_STAZ,
                               variable_x=COL_INFLAZIONE_STAZ,
                               max_lag_input=MAX_LAG_GRANGER)
    else:
        print(f"Skippo test Granger per '{COL_INFLAZIONE_STAZ}' -> '{COL_GT_INFLAZIONE_STAZ}' a causa di colonne mancanti.")


    # Test 3: indice_Tematico_GT_SA_diff1 -> NIC_destag_ISTAT_diff1
    if COL_GT_TEMATICO_STAZ in df_serie_stazionarie.columns and COL_INFLAZIONE_STAZ in df_serie_stazionarie.columns:
        print(f"\n*** Test di causalità per l'Indice GT Tematico verso Inflazione NIC ***")
        test_causalita_granger(df_serie_stazionarie,
                               variable_y=COL_INFLAZIONE_STAZ,
                               variable_x=COL_GT_TEMATICO_STAZ,
                               max_lag_input=MAX_LAG_GRANGER)
    else:
        print(f"Skippo test Granger per '{COL_GT_TEMATICO_STAZ}' -> '{COL_INFLAZIONE_STAZ}' a causa di colonne mancanti.")

    # Test 4: NIC_destag_ISTAT_diff1 -> indice_Tematico_GT_SA_diff1 (causalità inversa)
    if COL_GT_TEMATICO_STAZ in df_serie_stazionarie.columns and COL_INFLAZIONE_STAZ in df_serie_stazionarie.columns:
        print(f"\n*** Test di causalità inversa per l'Indice GT Tematico (Inflazione NIC verso Indice GT) ***")
        test_causalita_granger(df_serie_stazionarie,
                               variable_y=COL_GT_TEMATICO_STAZ,
                               variable_x=COL_INFLAZIONE_STAZ,
                               max_lag_input=MAX_LAG_GRANGER)
    else:
        print(f"Skippo test Granger per '{COL_INFLAZIONE_STAZ}' -> '{COL_GT_TEMATICO_STAZ}' a causa di colonne mancanti.")

    print("Analisi di causalità di Granger completata con successo.")
        
    print("\n>>> SCRIPT FASE 4 COMPLETATO <<<")
    

    


>>> INIZIO SCRIPT FASE 4: Analisi di Causalità di Granger (Corretto) <<<

--- Caricamento Serie Stazionarie da: /Users/tommaso/Desktop/tesi-inflation-gt/First_Difference_indexes/dati_preparati_fase2/indici_gt_nic_stazionari_fase2.csv ---
Serie caricate con successo. Shape: (252, 3)
--- Fine Caricamento ---


*** Test di causalità per l'Indice GT Inflazione verso Inflazione NIC ***
--- Test di Causalità di Granger: indice_Inflazione_GT_PCA_SA_diff1 -> NIC_destag_ISTAT_diff1 (max_lag=16) ---
Risultati per indice_Inflazione_GT_PCA_SA_diff1 Granger-causa NIC_destag_ISTAT_diff1:
  Lag 1: Causalità NON significativa (p-value F-test: 0.2341 > 0.05)
  Lag 2: Causalità NON significativa (p-value F-test: 0.2146 > 0.05)
  Lag 3: Causalità NON significativa (p-value F-test: 0.1558 > 0.05)
  Lag 4: Causalità SIGNIFICATIVA (p-value F-test: 0.0198 <= 0.05)
  Lag 5: Causalità SIGNIFICATIVA (p-value F-test: 0.0175 <= 0.05)
  Lag 6: Causalità SIGNIFICATIVA (p-value F-test: 0.0324 <= 0.05)
  Lag 7: Causa

/Users/tommaso/my_python_env/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/Users/tommaso/my_python_env/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/Users/tommaso/my_python_env/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/Users/tommaso/my_python_env/lib/python3.13/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
